# 3D metrics based on the surface distance

In [ ]:
import os
import sys
import glob

from pathlib import Path
import torch
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit import RDLogger, DataStructs
from rdkit.Chem import PandasTools
from rdkit.Chem.MolStandardize import rdMolStandardize
import copy


old_cwd = Path.cwd()
os.chdir(Path.cwd().parent)
from shape_alignment.molecule import Molecules, MoleculeInfo
os.chdir(old_cwd)

sys.path.append(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))))
from utils.chem_transforms import remove_atom_indices, replace_atom_indices

In [ ]:
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

In [ ]:
lg = RDLogger.logger()
lg.setLevel(RDLogger.CRITICAL)

## Load data

In [ ]:
pdb = '7JTO'
method = 'difflinker'

In [ ]:
gen_folder = 'data/generated'
poses_root = os.path.join(os.path.expanduser('~'), 'data/protacs/preprint')
filepath = glob.glob(os.path.join(gen_folder, f'{pdb}_sampled_{method}_valid_fil.csv'))
print(filepath)
df = pd.read_csv(filepath[0])
df.head()

### Get constrained embedded conformers

In [ ]:
pose_folders = os.path.join(poses_root, 'const_embed')
const_embed_path = os.path.join(pose_folders, f'{pdb}_{method}_embed')
const_embed_pose_folder = glob.glob(os.path.join(const_embed_path,'selected_min_confs'))
print(const_embed_pose_folder)

In [ ]:
def get_embedded_confs(sdfpath: str,) -> Chem.Mol:
    if sdfpath is np.nan:
        return None
    sdfpath = os.path.join(const_embed_pose_folder[0], sdfpath)
    mol = Chem.SDMolSupplier(sdfpath)[0]
    return mol

In [ ]:
# store embedded conformers and vinardo scores in dataframe using apply
df['embedded_mol'] = df.progress_apply(lambda x: get_embedded_confs(x['embedded_path']), axis=1)

In [ ]:
len(df[~df['embedded_mol'].isna()]), len(df)

### Get poses respective to method (shape aligned or generated)

In [ ]:
def get_shape_aligned_pose(sdffolder: str, mol_id: str) -> Chem.Mol:
    try:
        molpath = os.path.join(sdffolder, f'{mol_id}_pose.sdf')
        mol = Chem.SDMolSupplier(molpath)[0]
    except:
        try:
            molpath = os.path.join(sdffolder, f'{mol_id}_pose.mol')
            mol = Chem.MolFromMolFile(molpath)
        except:
            return None
    return mol

def get_diff_gen_pose(folder: str, sdfpath: str) -> Chem.Mol:
    if sdfpath is np.nan or sdfpath is None:
        return None
    sdffile = sdfpath.split('/')[-1]
    sdfpath_new = os.path.join(folder, pdb, sdffile)
    mol = Chem.SDMolSupplier(sdfpath_new)[0]
    return mol

In [ ]:
if method == 'difflinker':
    df_all = pd.read_csv(os.path.join(gen_folder, f'{pdb}_sampled_{method}_valid.csv'))
    gen_folder_path = os.path.join(poses_root, 'difflinker_gen_confs')
    df_all['method_mol'] = df_all.progress_apply(lambda x: get_diff_gen_pose(gen_folder_path, x['ori_gen_ptc_filename']), axis=1)

    col_insert = ['E_torsion',
    'clashes_cutoff',
    'clashes_vdw',
    'embedded_mol',
    'embedded_path',
    'rmsd_anc',
    'rmsd_wrh',
    'sc_rdkit',
    'vinardo']
    for col in col_insert:
        # transfer columns from df_all to df by mapping ID
        df_all[col] = df_all['ID'].map(df.set_index('ID')[col])
    df = df_all
else:
    df_all = pd.read_csv(os.path.join(gen_folder, f'{pdb}_sampled_{method}_valid.csv'))
    shape_align_path = os.path.join(gen_folder, f'{pdb}_{method}_aligned_poses')
    df_all['method_mol'] = df_all.progress_apply(lambda x: get_shape_aligned_pose(shape_align_path, x['ID']), axis=1)
    col_insert = ['E_torsion',
    'clashes_cutoff',
    'clashes_vdw',
    'embedded_mol',
    'embedded_path',
    'rmsd_anc',
    'rmsd_wrh',
    'sc_rdkit',
    'vinardo']
    for col in col_insert:
        # transfer columns from df_all to df by mapping ID
        df_all[col] = df_all['ID'].map(df.set_index('ID')[col])
    df = df_all

In [ ]:
len(df), len(df[~df['method_mol'].isna()]), len(df[~df['embedded_mol'].isna()])

### Get xtal poses

In [ ]:
xtal_folder = 'data/xtal_poses'
pdb_folder = os.path.join(xtal_folder, pdb, f'{pdb}_fragments')
xtal_protein_path = os.path.join(pdb_folder, f'{pdb}_protein.pdb')
# xtal_ext_linker = Chem.MolFromMolFile(os.path.join(pdb_folder,f'{pdb}_linker_extended.sdf'))
xtal_protac = Chem.MolFromMolFile(os.path.join(pdb_folder, f'{pdb}_protac.sdf'))
xtal_linker = Chem.MolFromMolFile(os.path.join(pdb_folder, f'{pdb}_linker.sdf'))
xtal_anchor = Chem.MolFromMolFile(os.path.join(pdb_folder, f'{pdb}_anchor.sdf'))
xtal_warhead = Chem.MolFromMolFile(os.path.join(pdb_folder, f'{pdb}_warhead.sdf'))

### Get linker only from poses

In [ ]:
def linker_from_extlinker(aligned_ext_linker, protac_smi, wrh_smi, anc_smi, linker_smi):
    if aligned_ext_linker is None:
        return None
    protac_mol = Chem.MolFromSmiles(protac_smi)
    wrh_mol = Chem.MolFromSmiles(wrh_smi)
    anc_mol = Chem.MolFromSmiles(anc_smi)
    linker_mol = Chem.MolFromSmiles(linker_smi)
    try:
        linker_mol = Chem.RemoveAllHs(linker_mol)
    except:
        return None
    match_ext_link= protac_mol.GetSubstructMatch(aligned_ext_linker)
    match_wrh = protac_mol.GetSubstructMatch(wrh_mol)
    match_anc = protac_mol.GetSubstructMatch(anc_mol)
    # get overlaps
    overlap_wrh = set(match_wrh).intersection(set(match_ext_link))
    overlap_anc = set(match_anc).intersection(set(match_ext_link))
    num_frag_wrh = len(overlap_wrh)
    num_frag_anc = len(overlap_anc)
    matches_linker = aligned_ext_linker.GetSubstructMatches(linker_mol)
    true_match = None
    for match in matches_linker:
    # sort match in reverse
        match = sorted(match, reverse=True)
        try:
            ext_frags = remove_atom_indices(aligned_ext_linker, match)
            # get individual fragments
            ext_frags = Chem.GetMolFrags(ext_frags, asMols=True)
            # check if number of atoms in ext_frags matches num_frag_wrh and num_frag_anc
            if len(ext_frags[0].GetAtoms()) == num_frag_wrh and len(ext_frags[1].GetAtoms()) == num_frag_anc:
                true_match = match
                break
            elif len(ext_frags[1].GetAtoms()) == num_frag_wrh and len(ext_frags[0].GetAtoms()) == num_frag_anc:
                true_match = match
                break
        except:
            continue
    if true_match is None:
        print('No match found')
        return None
    all_ext_lin_indices = [a.GetIdx() for a in aligned_ext_linker.GetAtoms()]
    to_keep = list(set(all_ext_lin_indices).intersection(set(true_match)))
    to_remove = list(set(all_ext_lin_indices).difference(set(true_match)))
    ev_indices = []
    for a in aligned_ext_linker.GetAtoms():
        a.SetAtomMapNum(a.GetIdx()+1)
    for b in aligned_ext_linker.GetBonds():
        if b.GetBeginAtomIdx() in to_remove and b.GetEndAtomIdx() in to_keep:
            ev_indices.append(b.GetBeginAtomIdx())
            to_remove.remove(b.GetBeginAtomIdx())
        elif b.GetEndAtomIdx() in to_remove and b.GetBeginAtomIdx() in to_keep:
            ev_indices.append(b.GetEndAtomIdx())
            to_remove.remove(b.GetEndAtomIdx())
    # sort in reverse
    to_remove = sorted(to_remove, reverse=True)
    linker_pose_aligned = remove_atom_indices(aligned_ext_linker, to_remove)
    a_dict = {a.GetAtomMapNum()-1: a.GetIdx() for a in linker_pose_aligned.GetAtoms()}
    ev_indices = [a_dict[i] for i in ev_indices]
    ev_indices = sorted(ev_indices, reverse=True)
    linker_pose_aligned = replace_atom_indices(linker_pose_aligned, ev_indices)
    for a in linker_pose_aligned.GetAtoms():
        a.SetAtomMapNum(0)
    for a in aligned_ext_linker.GetAtoms():
        a.SetAtomMapNum(0)
    try:
        linker_pose_aligned = Chem.RemoveHs(linker_pose_aligned)
    except:
        return linker_pose_aligned
    return linker_pose_aligned


In [ ]:
if method != 'difflinker':
    df['method_linker_mol'] = df.progress_apply(lambda x: linker_from_extlinker(x['method_mol'], x['protac_smiles'], x['warhead_smiles'], x['anchor_smiles'], x['linker_smiles']), axis=1)

In [ ]:
def linker_from_fullpose(protac_pose, wrh_smi, anc_smi):
    if protac_pose is None or protac_pose is np.nan:
        return None
    wrh_mol = Chem.MolFromSmiles(wrh_smi)
    anc_mol = Chem.MolFromSmiles(anc_smi)
    match_wrh = protac_pose.GetSubstructMatch(wrh_mol)
    match_anc = protac_pose.GetSubstructMatch(anc_mol)
    # sort match in reverse
    all_ptc_indices = [a.GetIdx() for a in protac_pose.GetAtoms()]
    to_remove = match_wrh + match_anc
    to_remove = list(set(to_remove))
    to_keep = list(set(all_ptc_indices) -set(to_remove))
    ev_indices = []
    for a in protac_pose.GetAtoms():
        a.SetAtomMapNum(a.GetIdx()+1)
    for b in protac_pose.GetBonds():
        if b.GetBeginAtomIdx() in to_remove and b.GetEndAtomIdx() in to_keep:
            ev_indices.append(b.GetBeginAtomIdx())
            to_remove.remove(b.GetBeginAtomIdx())
        elif b.GetEndAtomIdx() in to_remove and b.GetBeginAtomIdx() in to_keep:
            ev_indices.append(b.GetEndAtomIdx())
            to_remove.remove(b.GetEndAtomIdx())
    to_remove = sorted(to_remove, reverse=True)
    embedded_linker = remove_atom_indices(protac_pose, to_remove)
    a_dict = {a.GetAtomMapNum()-1: a.GetIdx() for a in embedded_linker.GetAtoms()}
    ev_indices = [a_dict[i] for i in ev_indices]
    ev_indices = sorted(ev_indices, reverse=True)
    embedded_linker = replace_atom_indices(embedded_linker, ev_indices)
    for a in embedded_linker.GetAtoms():
        a.SetAtomMapNum(0)
    for a in protac_pose.GetAtoms():
        a.SetAtomMapNum(0)
    # embedded_linker = Chem.RemoveHs(embedded_linker)
    return embedded_linker

In [ ]:
df['embedded_linker'] = df.progress_apply(lambda x: linker_from_fullpose(x['embedded_mol'], x['warhead_smiles'], x['anchor_smiles']), axis=1)

In [ ]:
if method == 'difflinker':
    df['method_linker_mol'] = df.progress_apply(lambda x: linker_from_fullpose(x['method_mol'], x['warhead_smiles'], x['anchor_smiles']), axis=1)

## Load model

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
root = os.path.dirname(os.path.dirname(os.getcwd()))
model = torch.load(os.path.join(root, "models/protacdb_extlinker_model_align.pth"), map_location=device)
model.to(device)
model.eval()

## Metrics

### Chamfer distance between new PROTAC conf and xtal conf
New PROTAC conf is embedded and/or directly generted (in case of difflinker)

Use to calculate also similarity ratio after reverse min max scaling over all samples.

In [ ]:
query_protac_xtal  = MoleculeInfo.from_sdf(os.path.join(pdb_folder, f'{pdb}_protac.sdf'))
df['cd_protac_embed2xtal'] = df.progress_apply(lambda x: query_protac_xtal.get_chamfer_distance(MoleculeInfo.from_rdkit_mol(x['embedded_mol']), device=device) \
                                               if x['embedded_mol'] is not None and x['embedded_mol'] is not np.nan else None, axis=1)
print('Chamfer distance between xtal and const embed: ', df['cd_protac_embed2xtal'].mean(skipna=True))

In [ ]:
query_linker_xtal  = MoleculeInfo.from_sdf(os.path.join(pdb_folder, f'{pdb}_linker.sdf'))
df['cd_linker_embed2xtal'] = df.progress_apply(lambda x: query_linker_xtal.get_chamfer_distance(MoleculeInfo.from_rdkit_mol(x['embedded_linker']), device=device) \
                                               if x['embedded_linker'] is not None and x['embedded_linker'] is not np.nan else None, axis=1)
print('Chamfer distance between xtal and const embed linker: ', df['cd_linker_embed2xtal'].mean(skipna=True))

In [ ]:
if method == 'difflinker':
    df['cd_protac_method2xtal'] = df.progress_apply(lambda x: query_protac_xtal.get_chamfer_distance(MoleculeInfo.from_rdkit_mol(x['method_mol']), device=device) \
                                               if x['method_mol'] is not None and x['method_mol'] is not np.nan else None, axis=1)
    print('Chamfer distance between xtal and generated pose: ', df['cd_protac_method2xtal'].mean(skipna=True))

    df['cd_linker_method2xtal'] = df.progress_apply(lambda x: query_linker_xtal.get_chamfer_distance(MoleculeInfo.from_rdkit_mol(x['method_linker_mol']), device=device) \
                                               if x['method_linker_mol'] is not None and x['method_linker_mol'] is not np.nan else None, axis=1)
    print('Chamfer distance between xtal and generated linker pose: ', df['cd_linker_method2xtal'].mean(skipna=True))

### Tanimoto similarity between new PROTAC and xtal reference

In [ ]:
# max tanimoto similarity to reference linker
PandasTools.AddMoleculeColumnToFrame(df, smilesCol='protac_smiles', molCol='mol_smi')
df.loc[:,'fp_ptc'] = df.mol_smi.progress_apply(Chem.RDKFingerprint)
ori_ptc_fp = Chem.RDKFingerprint(xtal_protac)
df['tanimoto_ptc'] = df.fp_ptc.progress_apply(lambda x: DataStructs.FingerprintSimilarity(x, ori_ptc_fp))

### Chamfer distance between methods
* Link-INVENT: between const. embed and shape aligned
* DiffLinker: between const. embed and generated pose

Based on: linker only

In [ ]:
df['cd_aligned_linker_embed2aligned'] = df.progress_apply(lambda x: MoleculeInfo.from_rdkit_mol(x['embedded_linker']).align_to_molecules2\
                                                          (Molecules.from_molecule_info([MoleculeInfo.from_rdkit_mol(x['method_linker_mol'])]), model).chamfer_distance \
                                                          if x['embedded_linker'] is not None and x['embedded_linker'] is not np.nan and \
                                                             x['method_linker_mol'] is not None and x['method_linker_mol'] is not np.nan else np.nan, axis=1)
print(df['cd_aligned_linker_embed2aligned'].mean(skipna=True))

In [ ]:
def fraction2threshold(df, threshold, column, above=True):
    '''
    Returns the fraction of molecules in a dataframe with a score above/below a given threshold
    '''
    df = df[~df[column].isna()]
    if above:
        df_above = df[df[column] >= threshold]
    else:
        df_above = df[df[column] <= threshold]
    metric = len(df_above)/len(df)*100
    return df_above, metric

In [ ]:
df_scored_fil, cutoff_fraction = fraction2threshold(df, 3.5, 'cd_aligned_linker_embed2aligned', above=False)
cutoff_fraction

In [ ]:
df_scored_fil, cutoff_fraction = fraction2threshold(df, 2.0, 'cd_aligned_linker_embed2aligned', above=False)
cutoff_fraction

In [ ]:
df_scored_fil, cutoff_fraction = fraction2threshold(df, 1.0, 'cd_aligned_linker_embed2aligned', above=False)
cutoff_fraction

### Save data

In [ ]:
df.columns

In [ ]:
df_save = df.drop(columns=['method_mol', 'embedded_mol', 'method_linker_mol', 'embedded_linker', 'mol_smi', 'fp_ptc'])

df_save.to_csv(os.path.join(gen_folder, f'{pdb}_sampled_{method}_valid.csv'), index=False)
df_fil_save = df_save[df_save['to_3d']]
df_fil_save.to_csv(os.path.join(gen_folder, f'{pdb}_sampled_{method}_valid_fil.csv'), index=False)